<a href="https://colab.research.google.com/github/leandrojgama/Extracao-de-dados-arquivos-de-PDF/blob/main/Extraindo_dados_de_PDF_de_OS_Calibra%C3%A7%C3%A3o_Qualifica%C3%A7%C3%A3o_git.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Referência:
* https://brayan-monserrah.medium.com/extraindo-informa%C3%A7%C3%B5es-de-arquivos-pdf-411b2c55c722



In [8]:
!pip install pdfplumber -q
#!pip install -U os
import pandas as pd
import os
import pdfplumber
import numpy as np


In [21]:
!wget "https://raw.githubusercontent.com/leandrojgama/Extracao-de-dados-arquivos-de-PDF/2fbfea4f9bf421046593e80efe5331b3e4207f07/MUF-00007.pdf"

--2023-11-17 16:47:30--  https://raw.githubusercontent.com/leandrojgama/Extracao-de-dados-arquivos-de-PDF/2fbfea4f9bf421046593e80efe5331b3e4207f07/MUF-00007.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17913 (17K) [application/octet-stream]
Saving to: ‘MUF-00007.pdf’

MUF-00007.pdf       100%[===================>]  17.49K  --.-KB/s    in 0s      

2023-11-17 16:47:30 (80.9 MB/s) - ‘MUF-00007.pdf’ saved [17913/17913]



In [27]:
arquivo='/content/MUF-00007.pdf'
pdf=pdfplumber.open(arquivo)

In [28]:
text=pdf.pages[1].extract_text().split('\n')
text

['Atributos',
 'Atributo Valor',
 'Critérios de aceitação:',
 'Temperatura: 600 ± 25°C',
 'Temperatura: 800 ± 25°C',
 'Qualificação de Desempenho',
 'Metodologia: 5 sensores calibrados devem ser distribuídos',
 'geometricamente no interior da câmara com carga e todos devem',
 'ser dispostos o mais próximo possível da carga. Os registros de',
 'temperatura devem ser adquiridos em intervalos de 01 minuto',
 'durante 5 horas. 100% das leituras pontuais deverão estar dentro do',
 'valor especificado.',
 'Critérios de aceitação:',
 'Temperatura: 600 ± 25°C',
 'Temperatura: 800 ± 25°C',
 'Observações:',
 'Set point da estufa: 600 °C e 800°C',
 'Para requalificação anual deve ser executada apenas qualificação de',
 'desempenho.',
 'Fonte: Farmacopéia Brasileira 5ª edição, USP 38.',
 'Prioridade Urgente',
 'Total de registros: 17',
 'Roteiro de aprovação',
 'Situação Ciclo Sequência Tipo Nome Prazo Aprovador Aprovação',
 'Henrique Marçal',
 '1 1 1',
 'Ricoy',
 'Rafael Henrique Rafael Henrique'

In [29]:
pdf=pdfplumber.open(arquivo)
text=pdf.pages[0].extract_text().split('\n')
req=False #'Requerimentos para Qualificação/Certificação'
requerimentos=''
for p in range(len(pdf.pages)):
  text=pdf.pages[p].extract_text().split('\n')
  atr=False
  for i in range(len(text)):
  #
    if req==False:
      cond='Objeto:'
      if text[i].find(cond)>-1:
        print(i,text[i+1])
      #
      if text[i].find('Atributos')>-1 and text[i+1].find('Atributo Valor')>-1:
          atr=True
        #
      if atr==True:
        cond='Intervalo de '
        if text[i].find(cond)>-1:
          print(i,text[i].split(cond)[-1].strip())

        cond='Requerimentos para '
        #
        if text[i].find(cond)>-1:
          req=True

          requerimentos+=text[i].replace(cond,'').strip()
          #print(requerimentos)
    #
    else:
      if text[i].find('Atributos')>-1 and text[i+1].find('Atributo Valor')>-1:
        atr=True

      #
      #print(i,text[i+1])

      if req==True and atr==True:
        #print(text[i])

        if (text[i].find('Prioridade Normal')>-1 or text[i].find('Prioridade Urgente')>-1) or text[i].find('Página 1 de')>-1:
          atr=False
        else:
          #print(text[i+1])
          requerimentos+='\n'+text[i].strip()



print(requerimentos)






13 MUF-00007 - FQ- Mufla Vulcan 1750 - SS-QLF - Solicitação de Serviços - Qualificação
33 Qualificação 1 ano
Qualificação/Certificação Para requalificação após alterações críticas deverá ser executada
Qualificação de Operação e Qualificação de Desempenho.
Qualificação de Operação
Metodologia: 5 sensores calibrados devem ser distribuídos
geometricamente no interior da câmara sem carga. Os registros de
temperatura devem ser adquiridos em intervalos de 01 minuto
durante 5 horas. 100% das leituras pontuais deverão estar dentro do
valor especificado.
Atributos
Atributo Valor
Critérios de aceitação:
Temperatura: 600 ± 25°C
Temperatura: 800 ± 25°C
Qualificação de Desempenho
Metodologia: 5 sensores calibrados devem ser distribuídos
geometricamente no interior da câmara com carga e todos devem
ser dispostos o mais próximo possível da carga. Os registros de
temperatura devem ser adquiridos em intervalos de 01 minuto
durante 5 horas. 100% das leituras pontuais deverão estar dentro do
valor especi

In [ ]:

os.listdir()

['.config', 'sample_data']

In [30]:
# @title Estração de dados de multiplo arquivos em PDF
os.chdir('/content')
arquivos=os.listdir()
data=pd.DataFrame()

for arq in arquivos:
  if arq[-3:]=='pdf':
    pdf=pdfplumber.open(arq)
    text=pdf.pages[0].extract_text().split('\n')
    req=False #'Requerimentos para Qualificação/Certificação'
    requerimentos=''
    dt={}
    dt['Arquivo']=arq
  #

    ##
    for p in range(len(pdf.pages)):
      text=pdf.pages[p].extract_text().split('\n')
      atr=False
      for i in range(len(text)):
      #
        if req==False:
          cond='Objeto:'
          if text[i].find(cond)>-1:
            dt[cond[:-1]]=text[i+1]

          #
          if text[i].find('Atributos')>-1 and text[i+1].find('Atributo Valor')>-1:
              atr=True
            #
          if atr==True:
            cond='Intervalo de '
            if text[i].find(cond)>-1:
              dt['Frequência']=text[i].replace('Intervalo de Qualificação','').replace('Intervalo de Calibração','').strip()

            cond='Requerimentos para '
            #
            if text[i].find(cond)==0:
              req=True

              requerimentos+=text[i].replace('Requerimentos para calibração','').replace('Requerimentos para Qualificação/Certificação','').strip()
              #print(requerimentos)
        #
        else:
          if text[i].find('Atributos')>-1 and text[i+1].find('Atributo Valor')>-1:
            atr=True

          #
          #print(i,text[i+1])

          if req==True and atr==True:
            #print(text[i])

            if text[i].find('Prioridade Normal')>-1 or text[i].find('Prioridade Urgente')>-1\
            or text[i].find('Prioridade Alta')>-1 or text[i].find('Prioridade Baixa')>-1\
            or text[i].find('Página 1 de')>-1:
              atr=False
            else:
              #print(text[i+1])
              requerimentos+=' '+text[i].strip()

    dt['Requerimentos']=requerimentos
    #print(dt)
    dt=pd.DataFrame([dt])
    data=pd.concat([data,dt],ignore_index=True)

In [ ]:
x=data['Objeto'].iloc[0]
data['Nome']=[i[12:i.find(' - SS-')] if type(i)==str else np.nan for i in data['Objeto'].values]
data['Ativo']=[i[:i.find(' - ')]if type(i)==str else np.nan for i in data['Objeto'].values]
data=data[['Ativo','Nome','Frequência','Requerimentos','Arquivo']]
data

,Ativo,Nome,Frequência,Requerimentos,Arquivo
0,GELBC-19C,Geladeira Panasonic MPR-1014-PA,1 ano,- Qualificação de desempenho (com carga) - Per...,GELBC-19C.pdf
1,FRE-00139,VM- Freezer Elber -30ºC / Corredor SVR,1 ano,- Para aprovação do equipamento no processo de...,FRE-00139.pdf
2,MUF-00004,MT- Mufla NDI,1 ano,Aceitabilidade de 10% no valor nominal. Solici...,MUF-00004.pdf
3,GEL-00094,BF- Refrigerador Laboratorial Thermo Ficher Sc...,1 ano,Qualificação Térmica com carga; Faixa de traba...,GEL-00094 QLF.pdf
4,ESI-00038,IB- Estufa Incubadora Tecnal TE-392/1 // Respo...,1 ano,- Os erros máximos admissíveis estão de acordo...,ESI-00038.pdf
...,...,...,...,...,...
301,FRE-00082,BF- Freezer Consul 260L,1 ano,Qualificação Térmica com carga; Faixa de traba...,FRE-00082.pdf
302,ESI-00053,BM- Estufa de Incubação Precision 3EG,1 ano,Faixa de uso: 23°C a 27°C (25°C ±2°C) Setpoint...,ESI-00053.pdf
303,GEL-00073,FQ- Geladeira Fanem 347 CVV/1,1 ano,Para requalificação anual deve ser executada a...,Parametro GEL-00073.pdf
304,GEL-00066,UG- Geladeira Científica Thermo Scientific,1 ano,- Faixa de uso: 2º a 8º - Utilizar no mínimo 1...,GEL-00066 QLF UG.pdf


In [ ]:
data.to_excel('Parametro.xlsx')